SQLALchemy 2 Scratchpad
===

Getting comfortable with SQLAlchemy 2.

https://docs.sqlalchemy.org/en/20/tutorial/index.html

In [1]:
import json
import os
import pickle
import random
import time
from datetime import datetime
from pathlib import Path
from pprint import pprint

import dotenv
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlalchemy
import supabase
from sqlalchemy import select
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy.sql import expression
from sqlalchemy.types import DateTime
from tqdm import tqdm

In [2]:
dotenv.load_dotenv("../.env")

True

In [3]:
# API test
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_API_KEY")
sb_client: supabase.Client = supabase.create_client(url, key)

In [4]:
data, count = sb_client.table("message").select("*").limit(10).execute()
count

2023-08-16 16:20:50,942:INFO - HTTP Request: GET https://bwerovdykjuxvfoozrge.supabase.co/rest/v1/message?select=%2A&limit=10 "HTTP/1.1 200 OK"


('count', None)

In [ ]:
data[1][0]

In [4]:
SUPABASE_CONNECTION_URL = os.environ["SUPABASE_CONNECTION_URL"]
SUPABASE_CONNECTION_URL[:22]

'postgresql://postgres:'

In [7]:
LEVON003_SUPABASE_CONNECTION_URL = os.environ["LEVON003_SUPABASE_CONNECTION_URL"]
LEVON003_SUPABASE_CONNECTION_URL[:22]

'postgresql://postgres:'

In [5]:
url = SUPABASE_CONNECTION_URL
engine = sqlalchemy.create_engine(url, echo=True)
with engine.begin() as conn:
    pass

2023-08-16 16:57:28,652 INFO sqlalchemy.engine.Engine select pg_catalog.version()


2023-08-16 16:57:28,652:INFO - select pg_catalog.version()


2023-08-16 16:57:28,654 INFO sqlalchemy.engine.Engine [raw sql] {}


2023-08-16 16:57:28,654:INFO - [raw sql] {}


2023-08-16 16:57:28,807 INFO sqlalchemy.engine.Engine select current_schema()


...

2023-08-16 16:57:28,961:INFO - [raw sql] {}


2023-08-16 16:57:29,111 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2023-08-16 16:57:29,111:INFO - BEGIN (implicit)


2023-08-16 16:57:29,113 INFO sqlalchemy.engine.Engine COMMIT


2023-08-16 16:57:29,113:INFO - COMMIT


In [9]:
table_name = "math_question_answer"
with engine.begin() as conn:
    result = conn.execute(sqlalchemy.text(f"select * from {table_name} limit 10"))
    results = result.all()
len(results)

2023-08-16 16:04:23,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2023-08-16 16:04:23,105:INFO - BEGIN (implicit)


2023-08-16 16:04:23,108 INFO sqlalchemy.engine.Engine select * from math_question_answer limit 10


2023-08-16 16:04:23,108:INFO - select * from math_question_answer limit 10


2023-08-16 16:04:23,109 INFO sqlalchemy.engine.Engine [generated in 0.00106s] {}


2023-08-16 16:04:23,109:INFO - [generated in 0.00106s] {}


2023-08-16 16:04:23,262 INFO sqlalchemy.engine.Engine COMMIT


2023-08-16 16:04:23,262:INFO - COMMIT


10

In [ ]:
results[0]

In [6]:
class Base(DeclarativeBase):
    pass

In [7]:
Base.metadata

MetaData()

In [8]:
class utcnow(expression.FunctionElement):
    type = DateTime()
    inherit_cache = True


@compiles(utcnow, "postgresql")
def pg_utcnow(element, compiler, **kw):
    return "TIMEZONE('utc', CURRENT_TIMESTAMP)"


class Generation(Base):
    __tablename__ = "rori_generative_api_generations"

    id: Mapped[int] = mapped_column(primary_key=True)
    created_at: Mapped[datetime] = mapped_column(DateTime, server_default=utcnow())
    generation: Mapped[str]
    context: Mapped[dict] = mapped_column(sqlalchemy.JSON)

In [9]:
Base.metadata.create_all(engine)

2023-08-16 16:57:40,590 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2023-08-16 16:57:40,590:INFO - BEGIN (implicit)


2023-08-16 16:57:40,596 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s


2023-08-16 16:57:40,596:INFO - SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s


2023-08-16 16:57:40,598 INFO sqlalchemy.engine.Engine [generated in 0.00155s] {'table_name': 'rori_generative_api_generations', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}


...

2023-08-16 16:57:40,758:INFO - 
CREATE TABLE rori_generative_api_generations (
	id SERIAL NOT NULL, 
	created_at TIMESTAMP WITHOUT TIME ZONE DEFAULT TIMEZONE('utc', CURRENT_TIMESTAMP) NOT NULL, 
	generation VARCHAR NOT NULL, 
	context JSON NOT NULL, 
	PRIMARY KEY (id)
)




2023-08-16 16:57:40,761 INFO sqlalchemy.engine.Engine [no key 0.00216s] {}


2023-08-16 16:57:40,761:INFO - [no key 0.00216s] {}


2023-08-16 16:57:40,877 INFO sqlalchemy.engine.Engine COMMIT


2023-08-16 16:57:40,877:INFO - COMMIT


In [10]:
# Base.metadata.reflect(bind=engine)
Base.metadata.tables["rori_generative_api_generations"]

Table('rori_generative_api_generations', MetaData(), Column('id', Integer(), table=<rori_generative_api_generations>, primary_key=True, nullable=False), Column('created_at', DateTime(), table=<rori_generative_api_generations>, nullable=False, server_default=DefaultClause(<__main__.utcnow at 0x16c0a3520; utcnow object>, for_update=False)), Column('generation', String(), table=<rori_generative_api_generations>, nullable=False), Column('context', JSON(), table=<rori_generative_api_generations>, nullable=False), schema=None)

In [24]:
Base.metadata.tables["rori_generative_api_generations"].drop(bind=engine)

2023-08-16 17:09:50,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2023-08-16 17:09:50,360:INFO - BEGIN (implicit)


2023-08-16 17:09:50,364 INFO sqlalchemy.engine.Engine 
DROP TABLE rori_generative_api_generations


2023-08-16 17:09:50,364:INFO - 
DROP TABLE rori_generative_api_generations


2023-08-16 17:09:50,365 INFO sqlalchemy.engine.Engine [no key 0.00126s] {}


2023-08-16 17:09:50,365:INFO - [no key 0.00126s] {}


2023-08-16 17:09:50,531 INFO sqlalchemy.engine.Engine COMMIT


2023-08-16 17:09:50,531:INFO - COMMIT


In [11]:
with sqlalchemy.orm.Session(engine) as session:
    new_gen = Generation(generation="Test text", context={"messages": []})
    session.add(new_gen)
    session.commit()

2023-08-16 16:58:05,948 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2023-08-16 16:58:05,948:INFO - BEGIN (implicit)


2023-08-16 16:58:05,952 INFO sqlalchemy.engine.Engine INSERT INTO rori_generative_api_generations (generation, context) VALUES (%(generation)s, %(context)s) RETURNING rori_generative_api_generations.id, rori_generative_api_generations.created_at


2023-08-16 16:58:05,952:INFO - INSERT INTO rori_generative_api_generations (generation, context) VALUES (%(generation)s, %(context)s) RETURNING rori_generative_api_generations.id, rori_generative_api_generations.created_at


2023-08-16 16:58:05,954 INFO sqlalchemy.engine.Engine [generated in 0.00158s] {'generation': 'Test text', 'context': '{"messages": []}'}


2023-08-16 16:58:05,954:INFO - [generated in 0.00158s] {'generation': 'Test text', 'context': '{"messages": []}'}


2023-08-16 16:58:06,111 INFO sqlalchemy.engine.Engine COMMIT


2023-08-16 16:58:06,111:INFO - COMMIT


In [17]:
with sqlalchemy.orm.Session(engine) as session:
    result = session.execute(select(Generation)).scalars().all()
result

2023-08-16 17:00:19,735 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2023-08-16 17:00:19,735:INFO - BEGIN (implicit)


2023-08-16 17:00:19,738 INFO sqlalchemy.engine.Engine SELECT rori_generative_api_generations.id, rori_generative_api_generations.created_at, rori_generative_api_generations.generation, rori_generative_api_generations.context 
FROM rori_generative_api_generations


2023-08-16 17:00:19,738:INFO - SELECT rori_generative_api_generations.id, rori_generative_api_generations.created_at, rori_generative_api_generations.generation, rori_generative_api_generations.context 
FROM rori_generative_api_generations


2023-08-16 17:00:19,740 INFO sqlalchemy.engine.Engine [cached since 26.3s ago] {}


2023-08-16 17:00:19,740:INFO - [cached since 26.3s ago] {}


2023-08-16 17:00:19,897 INFO sqlalchemy.engine.Engine ROLLBACK


2023-08-16 17:00:19,897:INFO - ROLLBACK


In [23]:
result[0].context

{'messages': []}